<a id="top"></a>

# Db2 Data Management Console RESTful APIs for SQL
This tutorial will explain some of the features that are available for Db2 when using the Db2 Data Management Console. The Db2 Data Management Console includes a number of RESTful services including those which allow for SQL execution. The services that are available through RESTful calls are outlined in the IBM Cloud API Documentation.

[RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html) 
    
There are a number of APIs available to use against Db2, but only three are of interest to us in this lab:
* Request an Access Token
* Execute an SQL statement
* Fetch the result set

The sections below will take you through the tasks required to use these APIs in a Python development environment.

## Connecting to Db2 with RESTful APIs
Connecting through RESTful APIs requires a different approach than a Db2 client. Rather than supply a userid and password, these APIs need an access token which is used for authentication. The access token is generated at the beginning of your session with Db2 and supplied with every RESTful call made to the database.

The second item that is required is the service url. This URL points to the server that will accept the RESTful request and then communicate with Db2 to return the results. Note that using Db2 on Cloud RESTful services is completely different from an OData gateway. OData RESTful gateways uses a specific syntax to communicate with the backend data source. In the case of these Db2 on Cloud APIs, the full SQL syntax is available for use.

The following information is required to use the RESTful APIs:
* **`Hostname`** - This is hostname or the IP address. 
* **`Port Number`** - 50000 is the Db2 port number.
* **`Userid`** - This is the userid that will be used to connect to the database to issue the SQL commands.
* **`Password`** - The password for the Db2 User.
* **`API`** - The service API.
* **`Service`** - The name of the service we want to use.

The **`API`** that is being used in this lab is `/dbapi/v4`. This service name can be different depending on what version of Db2 Data Management Console you are using. The **`Service`** depends on the request that we asking the API to provide. In this lab we are using two API calls:

* **`/auth/tokens`** - Provide a secure token to communicate with Db2 on Cloud
* **`/sql_jobs`** - Submit an SQL request to Db2
* **`/sql_jobs/#`** - Request results of the previous job that was submitted

### Load Db2 Connection Information
The first step is to load the Db2 connection information into this notebook. We don't need any of the Db2 `%sql` commands to run the RESTful calls. Instead, we are using the following information to connect to the RESTful service.

In [ ]:
db2id = {
 'db': 'SAMPLE',
 'hostname': 'localhost',
 'https_url': 'http://localhost:11080',
 'password': 'db2inst1',
 'port': 50000,
 'username': 'DB2INST1',
 'access_token' : ''
}

After getting the connection information for the database, there are three pieces of information required to set up the access token:
* **`https_url`** - The url of the server that is running Db2 on Cloud
* **`username`** - The userid that you used to connect to the Db2 database
* **`password`** - The password associate with your userid

Once an access token has been established, the userid/password is no longer required.

### Load Sample Data
In order to run some of these examples, we are going to connect to the Db2 locally and generate the `EMPLOYEE` and `DEPARTMENT` tables.

In [ ]:
%run db2.ipynb
%sql CONNECT CREDENTIALS db2id
%sql -sampledata

Next we generate 500 rows of customer data (in JSON format) that we will use to populate some tables.

In [ ]:
%run generate_customer_data.ipynb

The next step is to create our base table that will hold all of the JSON data.

In [ ]:
%%sql -q
  DROP TABLE CUSTOMERS;
  CREATE TABLE CUSTOMERS 
    (
    CUSTNO  INT NOT NULL,
    DETAILS VARCHAR(2048)
    );

The next piece of code will import the data into Db2. The Python code will read the `customer.js` file, parse it and then insert in into the `CUSTOMERS` table. Note that the entire JSON record is stored into the `DETAILS` column, while the `CUSTNO` column is set to the `customerid` field that is found in the JSON record. This load step will take a few minutes to complete because of the network time.

In [ ]:
import io
import json
print("Starting Load")
%sql autocommit off
x = %sql prepare INSERT INTO CUSTOMERS VALUES (?,?)
if (x != False):
    i = 0
    with open("customers.js","r") as records:
        for record in records:
            rec = json.loads(record)
            custno = rec['customerid']
            i += 1
            rc = %sql execute :x using custno@int, record@char
            if (rc == False): break
            if ((i % 10) == 0): 
                print(".",end='')
                %sql commit hold
                
    %sql commit work  
%sql autocommit on
print('')
print('Loaded completed')

Once the 500 rows are loaded, we can create some traditional Db2 tables from the JSON by using the new Db2 ISO JSON functions.

## Converting JSON into Relational Tables
While we could do all of the following examples using native JSON calls, it makes it easier to visualize the data if we convert it to traditional relational tables. The `TRANSACTIONS` table will contain all of the original JSON data that we want to convert.

In [ ]:
%%sql -q
DROP TABLE TRANSACTIONS;
CREATE TABLE TRANSACTIONS
  (
  TX_DATE DATE,
  CITY VARCHAR(32),
  STATE CHAR(2),
  ZIPCODE INT,
  PRODUCT_NAME VARCHAR(32),
  PRODUCT_QTY INT,
  PRODUCT_COST DEC(7,2),
  CCARD CHAR(4)
  );

To simplify the unnesting of arrays, we will include the JSON array functions that take the JSON arrays and generate a table of values.

In [ ]:
%run JSON_Array_Functions.ipynb

Next we generate the rows we need to populate the transactions table.

In [ ]:
%%sql
INSERT INTO TRANSACTIONS
  SELECT 
      JSON_VALUE(P.VALUE,  '$.tx_date'           RETURNING DATE),
      JSON_VALUE(C.DETAILS,'$.contact.city'      RETURNING VARCHAR(32)),
      JSON_VALUE(C.DETAILS,'$.contact.state'     RETURNING CHAR(2)), 
      JSON_VALUE(C.DETAILS,'$.contact.zipcode'   RETURNING CHAR(5)),
      JSON_VALUE(P.VALUE,  '$.product'           RETURNING VARCHAR(32)),
      JSON_VALUE(P.VALUE,  '$.quantity'          RETURNING INTEGER),
      JSON_VALUE(P.VALUE,  '$.item_cost'         RETURNING DEC(7,2)),            
      JSON_VALUE(C.DETAILS,'$.payment.card_type' RETURNING CHAR(4))
  FROM CUSTOMERS C,
       TABLE( JSON_ARRAY_OBJECT(DETAILS,'$.purchases') ) AS P
  WHERE JSON_EXISTS(C.DETAILS,'$.purchases') IS TRUE 

In [ ]:
%sql select count(*) from transactions

In [ ]:
%sql -grid select * from transactions

### Calling Restful Services

There are two types of RESTful calls that are used with Db2 on Cloud:

* **`GET`** - Get results from a SQL request
* **`POST`** - Request an Access token, or issue an SQL command

All RESTful calls require the host IP address and the service URL:
* **`host`** - This is the IP address of the machine that is hosting Db2 on Cloud
* **`api`** - The API library that is being used to communicate with Db2 on Cloud
* **`service`** - The service (API) that is being requested. 

As mentioned previously, the API library that is used for Db2 on Cloud is `/dbapi/v3`. The full URL has the following format:

```
https:// + host + api + service
```

If your host was xyz.abc.com and we are using the `/auth/tokens` service, the full URL would be:

```
https://xyz.abc.com/dbapi/v3/auth/tokens
```

This URL is then placed into a RESTful call that has one of the following formats:

```
RESTful.call( host + api + service, json=data)
RESTful.call( host + api + service, headers=tokens, json=data)
```

The first form of the RESTful call sends parameters via the **`json=data`** keyword. This format of RESTful call is used to generate an access token. The second form of the RESTful call will supply parameters through the **`json=data`** keyword, and also supply the access token via the **`headers=tokens`** keyword. Any SQL request needs this access token to be supplied, otherwise the Db2 on Cloud service will not be able to authenticate the RESTful call.

### Generating an Access Token
The first step in generating an access token is to build the URL for the service. From a programming perspective, the host URL will be combined with the API library to form the stub for all future RESTful URL requests. The HOST IP address is derived from the Db2 connect parameters.

In [ ]:
api = '/dbapi/v4'
host = db2id['https_url'] + api
print( "Host URL = " + host)

Next we need to build the JSON parameter structure required for the authentication call. This JSON parameter needs to contain the following information:
```json
{
    "userid" : "xyz12345",
    "password" :"randompassword"
}
```
The next statement will create a Python variable that contains your userid and password from in the Db2 connection.

In [ ]:
userinfo = {
    "userid" : db2id['username'],
    "password" : db2id['password']
}

The final step before we issue the RESTful call is to load the Python RESTful library into the notebook and define what service we are going to use in our RESTful call.

In [ ]:
import requests
service = "/auth/tokens"

The final format of the RESTful call to request an access token looks like this:
```
RESTful.post(host + service,json=userinfo)
```
If this call returns successfully, the `post` request will return a structure that looks like this:
```json
{
  "userid": "STRING",
  "token": "STRING"
}
```
Normally the `requests` call would be assigned to a variable, i.e `x = requests.post()`. A quick test to see if the RESTful call was successful is to compare `x.ok` to `True`. It is also good coding practice to surround the RESTful calls in a `try/except` block to catch any major errors that may occur.

Execute the next statement to retrieve an access token.

In [ ]:
r = requests.post(host + service, json=userinfo)

We check to see if the results of the call are okay. `OK` means that an HTTP return code of `200` was returned from the service which means that a valid call was made and that it was successful. If the HTTP return code is anything else, you would get a value of `False`.

In [ ]:
r.ok

The other option is to view the HTTP return code. Some of the possible values are:
* 400 - Bad request which means the parameters sent were incorrect or missing
* 401 - Unauthorized request
* 403 - Forbidden - Probably the wrong service or URL
* 404 - Not found - Probably the wrong service or URL

The `401` code is returned from the `/auth/tokens` service when an incorrect userid or password are sent to it. The previous call succeeded, so we can check the return code using `r.status_code`. 

In [ ]:
r.status_code

Since the status code is `200`, we can examine the results returned by the RESTful call by checking the json structure in the answer set since this will contain the access token.

In [ ]:
r.json()

Now we can assign the access token to a variable that we will use for all of the remaining calls. 

In [ ]:
access_token = r.json()['token']

The access token is valid for about an hour which means that you will need to refresh it if you are starting off in the middle of a notebook after having been away for a while. The next statement will create some invalid credentials and display the RESTful code and the JSON error structure that is returned.

In [ ]:
userinfo_bad = {
    "userid" : "unknown",
    "password" : "password"
}
r = requests.post(host + service, json=userinfo_bad)
print(r.status_code)
print(r.json())

### Request Access Token Routine
To simplify the process of getting an access token, the following Python routine will take the `db2id` structure and return an access token. In the event of an error, the routine will print the error message and return `None`. Note that the API library is imbedded in the routine so that if the API changes, the code must be updated.

In [ ]:
def get_token(db2id):
    
    import requests
    
    # Set up structures required for the RESTful call
    api = '/dbapi/v4'
    service = "/auth/tokens"
    host = db2id['https_url'] + api
    userinfo = {
        "userid" : db2id['username'],
        "password" : db2id['password']
    }
    
    # Call the RESTful service
    try:
        r = requests.post(host + service, json=userinfo)
    except Exception as err:
        print("RESTful call failed. Detailed information follows.")
        print(err)
        return None
    
    # Check for Invalid credentials
    if (r.status_code == 401): # There was an error with the authentication
        print("RESTful called failed.")
        message = r.json()['message']
        print(message)
        return None
    
    # Check for anything other than 200/401
    if (r.status_code != 200): # Some other failure
        print("RESTful called failed. Detailed information follows.")
        print(r.json())
        return None
    
    # Retrieve the access token
    try:
        access_token = r.json()['token']
    except:
        print("RESTful call did not return an access token.")
        print(r.json())
        return None
    
    return access_token

From a convenience perspective, we can add the access token to the Db2 credentials that we have. This way we only need to pass the Db2id to other routines rather than having the access token and the Db2id.

In [ ]:
db2id['access_token'] = get_token(db2id)

### Executing SQL Statements
There are two steps required to access data from Db2 on Cloud. The first RESTful call will post the SQL statements to Db2 on Cloud to be executed. It is possible to run multiple statements at once, but for the purposes of this lab we will only submit one statement at a time. 

The structure of the SQL request is:
```json
{
    "commands" : "sql",
    "limit" : x,
    "separator" : ";",
    "stop_on_error" : "yes"
}

```
The parameters are set as follows:

* commands - The SQL script to be executed (could be multiple statements).
* limit - Maximum number of rows that will be fetched for each result set.
* separator - SQL statement terminator. A character that is used to mark the end of a SQL statement when the provided SQL script contains multiple statements.
* stop_on_error - If 'yes', the job stops executing at the first statement that returns an error. If 'no', the job continues executing if one or more statements returns an error.

The service that is called to execute SQL statements is `/sql_jobs`. The RESTful post request has following format:
```
request.post(host + service, headers=authtoken, json=sql)
```
The `headers=authtoken` contains the authentication token that was retrieved in the first step and `json=sql` refers to the SQL statement and parameters required to execute it.

The value that is returned from the service is:
```json
{
  "id": "STRING",
  "commands_count": "INTEGER",
  "limit": "INTEGER"
}
```
The important piece of information is the `id` field which we need to track the execution of the SQL command(s). The first step that we need to take is to create the authentication header. The header has the format:
```json
{'Authorization': "Bearer " + token, 'X-DB-Profile': profile_name}
```
The format may look strange but this is the standard for passing tokens between a client and server. The `profile_name` is defined as part of the Db2 Console, which in this case just translates to the database name.

The next statement will create the authentication header.

In [ ]:
auth_header = {
    "Authorization" : "Bearer " + db2id['access_token'], 'X-DB-Profile': db2id['db']
}

Now we can create an SQL command with the associated parameters.

In [ ]:
sql_command = {
    "commands" : "SELECT COUNT(*) FROM EMPLOYEE",
    "limit" : 1000,
    "separator" : ";",
    "stop_on_error" : "yes"
}

At this point we have enough information to request that the SQL be executed for us. We need to supply the authentication token and the SQL request.

In [ ]:
service = "/sql_jobs"
r = requests.post(host + service, headers=auth_header, json=sql_command)

In [ ]:
r.json()

In [ ]:
sql_command

We will check the return code (201) and if it is okay we will display the json results that are returned.

In [ ]:
if r.status_code == 201:
    print(r.json())
else:
    print("Something went wrong with the call!")

The `id` field is what we need to keep track of. The `id` is used to retrieve the answer set(s) that are associated with the SQL call. The value will be saved into a Python variable for future use.

In [ ]:
jobid = r.json()['id']

### Retrieving an Answer Set
Now that the SQL statement has been sent off for execution, we must request the results. The RESTful API is exactly the same as requesting the SQL to be run, but you need to add the job id to the end of the service. Instead of using a `POST` request we use `GET`. The format of the command is:
```
request.get(host + service + "/" + jobid, headers=authtoken)
```
When the call is successful, the result contains the following information:
```json
{
  "id": "STRING",
  "status": "STRING",
  "results": {
    "command": "STRING",
    "columns": "ARRAY",
    "rows": "ARRAY",
    "rows_count": "INTEGER",
    "limit": "INTEGER",
    "last_inserted": "INTEGER",
    "rows_affected": "INTEGER",
    "runtime_seconds": "DOUBLE",
    "error": "STRING"
  }
}
```
The `status` field contains either `completed`,`running`, or `failed`. It is possible that you only get an intermediate result set (perhaps because the answer set is still being gathered) so `running` will be returned. When the status is `running` you may already have some data in the `results` field. You must retrieve this data before issuing another request. The data that is returned is not cumulative which means any results that are returned in the RESTful call are lost on the next call. A general strategy would be to write code similar to:
```
r = restful.get(...)
if (r.status == "failed") leave...
rows = get rows from r
while r.status == "running"
    r = restful.get(...)
    rows = get rows from r
```
Since we only requested a count of rows, the following code should be successful (200).

In [ ]:
r = requests.get(host + service + "/" + jobid,headers=auth_header)
if r.status_code == 200:
    print(r.json())
else:
    print("Something went wrong with the call!")

The column names and rows of the result set are found in the 'results' field and the rows are represented as an array of values. To extract the column names use the following syntax.

In [ ]:
results = r.json()['results']
columns = results[0]['columns']
rows = results[0]['rows']
print('Column Names')
print(columns)
print('Rows')
print(rows)

The column names are in a single-dimensional array, while the rows are a 2-dimensional array. If you are using pandas dataframes to manipulate data, you can convert the array values using the following syntax. The last statement converts the character columns into numeric values. The RESTful call returns the answer set as strings so this step makes it easier to plot data.

In [ ]:
import pandas
df = pandas.DataFrame(data=rows,columns=columns)
cols = df.columns
df[cols] = df[cols].apply(pandas.to_numeric, errors='ignore')

Now that we have the data in dataframe format, just referring to the variable will print it in a nicer format.

In [ ]:
df

For a more sophisticated sample, we will request all of the table names in the SYSCAT.TABLES table.

In [ ]:
sql = 'select tabname, card, npages, fpages from syscat.tables fetch first 10 rows only'
sql_command = {
    "commands" : sql,
    "limit" : 1000,
    "separator" : ";",
    "stop_on_error" : "yes"
}
service = "/sql_jobs"
r = requests.post(host + service, headers=auth_header, json=sql_command)

Once the request is done we can check the status code and the retrieve the data. One thing to be aware of is that if the answer set is not available immediately, you still get a successful return code (200), but the return result will be empty and the status will be set to `running`. 

In [ ]:
if (r.status_code == 201):
    jobid = r.json()['id']
    r = requests.get(host + service + "/" + jobid,headers=auth_header)
    if (r.status_code == 200):
        results = r.json()['results']
        columns = results[0]['columns']
        rows = results[0]['rows']
        df = pandas.DataFrame(data=rows,columns=columns)
        cols = df.columns
        df[cols] = df[cols].apply(pandas.to_numeric, errors='ignore')
        display(df)
    else:
        print("Things didn't work out")
else:
    print("Error on first POST request")

### Simplifying Data Retrieval
To make the process of getting an answer back easier, the following code will take all of the previous steps and encapsulate them into a single call. The function has the following format:
```
restSQL(db2id,sql)
```
The function will return a pandas dataframe as a result set. The parameters are:
* db2id - The id information from the Db2 Cloud service which includes the access_token created earlier
* sql - The SQL to be executed (single statement only)

One of the issues when using access tokens is that they expire after a certain period of time. This makes the access more secure since a token will expire in the event that it was accidentially released externally. However, this means that your queries may not work after a period of time because of an expired access token. If you do get an error with the SQL call, you will need to request a new access token with the following call:
```python
db2id['access_token'] = get_token(db2id)
```
The code in this function also checks to make sure that the SQL has completed execution. There is a possibility that the SQL will run for a long period of time so the function needs to continuously poll the server with RESTful calls until the SQL has completed.

In [ ]:
def restSQL(db2id,sql):
    
    import requests
    import pandas
    
    api = '/dbapi/v3'
    host = db2id["https_url"] + api
    
    authid = {"Authorization" : "Bearer " + db2id['access_token'], 'X-DB-Profile': db2id['db']}
    
    # Step 1: Build the SQL query and get the Job ID
    
    service = "/sql_jobs"

    request = {
        "commands" : sql,
        "limit" : 10000,
        "separator" : ";",
        "stop_on_error" : "yes"
    }
    
    try:
        r = requests.post(host + service,headers=authid,json=request)
        if (r.ok == False):
            print("SQL Execution error")
            print(r.json())
            return None
        
    except Exception as err:
        print("Error attempting RESTful call")
        print(err)
        return None
    
    jobid = r.json()['id']
    
    # Step 2: Wait for Answer set to complete
    
    service = "/sql_jobs/" + jobid
    rows = None
    while True:
        try:
            r = requests.get(host + service,headers=authid)
            if (r.ok == False):
                print("SQL answer set retrieval failed.")
                print(r.json())
                return None
            
        except Exception as err:
            print("Error attempting RESTful call")
            print(err)
            return None
        
        status = r.json()['status']
        if (status == "failed"):
            print("SQL failed")
            print(r.json())
            return None
                
        results = r.json()['results']
        try:
            columns = results[0]['columns']
            if (rows == None):
                rows = results[0]['rows']
            else:
                rows = rows + results[0]['rows']
        except:
            None
            
        if (status == 'completed'): break
            
    # Step 3: Create the Dataframe to return to the application
                    
    df = pandas.DataFrame(data=rows,columns=columns)
    cols = df.columns
    df[cols] = df[cols].apply(pandas.to_numeric, errors='ignore')
    
    return df

Now we can execute the previous SQL example with just a simple call.

In [ ]:
sql = 'select tabname, card, npages, fpages from syscat.tables fetch first 10 rows only'
restSQL(db2id,sql)

### Plot Routine
This code will produce a graph of the results from an SQL statement. This routine allows more flexibility on what is displayed on the chart, including changes the colors, the titles, and the axis names.

The format of the Plot is:
```
plotSQL(db2id, sql "Title of the plot", "X Axis Title", "Y Axis Title", rotation, color palette)
```
If Rotation is set to True, the x axis labels will be rotated by 30 degress and place the text at a slight angle to make longer text easier to read. The color palette is either m1, m2, m3, or m4.

In [ ]:
import matplotlib.pyplot as plt

m1 = ('#009926','#00994d','#009973','#009999','#007399','#004d99','#002699','#000099')
m2 = ("#00ff40","#00ff80","#00ffbf","#00ffff","#00bfff","#0080ff","#0040ff")
m3 = ("#9933ff","#cc33ff","#ff33ff","#ff33cc","#ff3399","#ff3366","#ff3333")
m4 = ("#ff00bf","#ff0080","#ff0040","#ff0000","#ff0000","#ff4000","#ff8000","#ffbf00","#ffff00")

def plotSQL(db2id, sql, title, xaxis, yaxis, rotation, colormap):
    if (rotation == True):
        rot = 30
    else:
        rot = 0
    df = restSQL(db2id, sql)
    if (df is None):
        print("RESTful call failed")
        return
    xvalues = df.columns.values[0]
    yvalues = df.columns.values[1]
    df.plot(kind='bar',x=xvalues,y=yvalues,color=colormap);
    _ = plt.xlabel("", fontsize=12);
    _ = plt.ylabel(yaxis, fontsize=12);
    _ = plt.suptitle(title, fontsize=20);
    _ = plt.xticks(rotation=rot);
    _ = plt.plot();

Now execute each one of these SQL statements to get graphs and output about the status of the company.

### Sales by State

In [ ]:
sqlin = \
'''
WITH 
SALES(state, amount) AS
   ( 
   SELECT STATE, SUM(PRODUCT_QTY * PRODUCT_COST) 
   from TRANSACTIONS 
   GROUP BY STATE
   ) 
SELECT state, sum(amount) from SALES 
GROUP BY state 
ORDER BY 2 DESC FETCH FIRST 10 ROWS ONLY
WITH UR
'''
plotSQL(db2id, sqlin, 
    "Sales per State", 
    "State",
    "Revenue",
    False,
    m1)

### Products Ordered

In [ ]:
sqlin = \
'''
WITH 
  SALES(product, quantity) AS ( 
      SELECT PRODUCT_NAME, PRODUCT_QTY 
      from TRANSACTIONS
      ) 
SELECT product, sum(quantity) from SALES 
GROUP BY product
ORDER BY 2 DESC
FETCH FIRST 10 ROWS ONLY
WITH UR
'''
plotSQL(db2id, sqlin, 
    "Products Sold", 
    "Product",
    "Quantity",
    True,
    m2)

### Weekly Sales with Moving Average

In [ ]:
sqlin = \
"""
WITH TOTALPROD(TX_DATE, TX_TOTAL) AS 
  (
  SELECT WEEK(TX_DATE), SUM(PRODUCT_QTY*PRODUCT_COST) FROM TRANSACTIONS
      WHERE YEAR(TX_DATE) = 2017 AND WEEK(TX_DATE) <= 52
  GROUP BY WEEK(TX_DATE)
  )
SELECT TX_DATE, TX_TOTAL AS SALES, 
     AVG(TX_TOTAL) OVER (
       ORDER BY TX_DATE
     ROWS BETWEEN 8 PRECEDING AND CURRENT ROW) AS MOVING_AVG
  FROM TOTALPROD
  ORDER BY TX_DATE
"""
df = restSQL(db2id, sqlin)
txdate= df['TX_DATE']
sales = df['SALES']
avg = df['MOVING_AVG']

plt.xlabel("Week of Year", fontsize=12);
plt.ylabel("Sales", fontsize=12);
plt.suptitle("Weekly Sales and Moving Average", fontsize=20);
plt.plot(txdate, sales, 'r');
plt.plot(txdate, avg, 'b');
plt.show();

### Top Products Sales by State

In [ ]:
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
sqlin = \
"""
WITH 
  SALES(product, state, quantity) AS ( 
      SELECT PRODUCT_NAME, STATE, SUM(PRODUCT_QTY)
      from TRANSACTIONS
      GROUP BY PRODUCT_NAME, STATE
      ),
  MAXSALES(state, total) AS (
      SELECT STATE, MAX(quantity)
      FROM SALES
      GROUP BY STATE
  )
SELECT s.state, s.product, m.total 
  FROM SALES s, MAXSALES m
WHERE
  s.state = m.state and
  s.quantity = m.total
ORDER BY m.total desc
FETCH FIRST 8 ROWS ONLY
"""
result = restSQL(db2id, sqlin)
result.style.background_gradient(cmap=cm)

## Summary
Using RESTful calls to Db2 using the Db2 Data Management Console removes much of the complexity of communicating with the database. There are no drivers required, no configuration file, nor any administration required on the client that is communicating with the database. All communication is done using RESTful API calls, which are available on all browsers and all operating systems. The calls to the database are replaced with standard POST and GET requests. Enabling RESTful support to Db2 opens up the type of applications that you can write and clients that you can connect to Db2 with.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]